In [7]:
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
import pandas as pd
import json
import create_directory

# Prepare data

Ta sẽ sử dụng các feature vector được trích xuất bằng Openface (Xem kĩ readme step_by_step_face_recognition để biết thêm thông tin), ta sẽ chỉ xem xét trên tập train

In [8]:
feature_path = os.path.join(create_directory.marvel_data_dir, "feature/Train/hog_openface")
with open(os.path.join(create_directory.recognition_model_dir, 'label2idx.json')) as json_file:
    label2idx = json.load(json_file)
idx2label = {y: x for x, y in label2idx.items()}
label_idx = []
feature = []
for i in os.listdir(feature_path):
    if i not in ["Unknown", ".DS_Store"]:
        for j in os.listdir(os.path.join(feature_path, i)):
            feat=np.load(os.path.join(os.path.join(feature_path, i), j))
            feature.append(feat.reshape(128).tolist()) 
            label_idx.append(label2idx[i])
    else:
        continue

# PCA and plot

Giảm chiều dữ liệu từ 128 chiều xuống 3 chiều

In [9]:
pca = PCA(n_components=3)
feature_pca = pca.fit_transform(feature)
feature_pca.shape

(1105, 3)

In [10]:
coord_info = {'x':[], 'y':[], 'z':[], 'label':[]}
for feat, idx in zip(feature_pca, label_idx):
    coord_info['x'].append(feat[0])
    coord_info['y'].append(feat[1])
    coord_info['z'].append(feat[2])
    coord_info['label'].append(idx2label[idx])
info = pd.DataFrame.from_dict(coord_info)
fig = px.scatter_3d(info, x='x', y='y', z='z',
              color='label')
fig.show()

# Clustering

In [11]:
km = KMeans(
    n_clusters=6, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)

predict_label_idx = km.fit_predict(feature)

In [12]:
clustering_result = pd.DataFrame({"predict": predict_label_idx, "ground_truth": label_idx})
clustering_result

,predict,ground_truth
0,1,3
1,1,3
2,1,3
3,1,3
4,1,3
...,...,...
1100,3,5
1101,3,5
1102,3,5
1103,3,5


In [13]:
clustering_result.to_csv(os.path.join(create_directory.result_dir, "clustering_result.csv"), index=False)

Sau khi sử dụng PCA và Kmean, ta nhận thấy openface feature vector có khả năng phân tách cụm khá tốt nhờ sử dụng triplet loss